In [2]:
from tensorflow.keras import optimizers
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

import tools
import cv2
import numpy as np
import pickle
import random
import glob

In [3]:
# Fixed for Cats & Dogs color images
CHANNELS = 3

IMAGE_RESIZE = 224 # Original weights
RESNET50_POOLING_AVERAGE = 'avg'

model = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5 
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is 
model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = "imagenet"))


In [4]:
# [256, 256, 128, 32 ] --> input activation_unit_list
def get_model(activation_units_list, add_normalization):
    model = Sequential()
    model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = "imagenet"))
    # Say not to train first layer (ResNet) model as it is already trained
    model.layers[0].trainable = False
    for activation_unit in activation_units_list:
        model.add(Dense(activation_unit, activation="relu"))
        if add_normalization:
            model.add(BatchNormalization())
    model.add(Dense(4, activation="relu"))
    model.compile(optimizer="adam", loss="mse", metrics=["mse"])
    return model

In [5]:
root_paths =glob.glob("cat-dataset/*", recursive=True)
paths = []
# Lee el path de las imágenes y los guarda en paths
for temp_root_path in root_paths:
    temp_paths = tools.retrieve_img_file_paths(f"{temp_root_path}/*")
    for path in temp_paths:
        paths.append(path)
X = []
Y = []
for path in paths:
    img = cv2.imread(path)
    img_resized = cv2.resize(img, (64, 64))  # Tuning
    X.append(img_resized)
    points = tools.retrieve_face_coordinates(path)
    left_eye = tools.get_resized_eye_point_position(img, img_resized, points["left-eye"])
    right_eye = tools.get_resized_eye_point_position(img, img_resized, points["right-eye"])
    cc = [left_eye[0], left_eye[1], right_eye[0], right_eye[1]]
    Y.append(cc)

X = np.array(X)
Y = np.array(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [6]:
# Tuning
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [7]:
# https://keras.io/preprocessing/image/

# Tuning
epochs = 10
batch = 128
# Creates a resnet50 with an additional layer to train with cat eyes position
# archs = [[128, 128, 32], [64, 128, 128, 32, 16], [128, 128, 32, 16]]

archs = [[128, 128, 32]]

for arch in archs:
    for norm in [True, False]:
        mm = get_model(arch, norm)
        history = mm.fit_generator(datagen.flow(X_train, y_train, batch_size=batch), steps_per_epoch=len(X_train) / batch, epochs=epochs)
        arch_str = str(arch)
        file_name = f"train_history/{arch_str}_{norm}_epoch{epochs}_batch{batch}"

        print(f"saving to: {file_name}")
        with open(file_name, "wb") as f:
            pickle.dump(history.history, f) 

        mm.save(f"{file_name}.model")


Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 54.6640625 steps
Epoch 1/10


/home/jota/.virtualenvs/crp/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/jota/.virtualenvs/crp/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/jota/.virtualenvs/crp/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/jota/.virtualenvs

 1/54 [..............................] - ETA: 3:14

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_1/resnet50/conv1_conv/Conv2D (defined at <ipython-input-7-9173ee53224e>:14) ]] [Op:__inference_distributed_function_26316]

Function call stack:
distributed_function
